In [1]:
import pandas as pd
import json
import random

In [2]:
df = pd.read_csv('../../data/mimic-acute-mi.csv')
df.head()

,hadm_id,ecg_time,st_elevation,st_depression,t_wave,Acute_MI,study_id,text,valuenum,comments,troponin,STEMI,NSTEMI
0,29196424.0,2087-05-08 00:05:00,0,0,0,0,44673611,Allergies: \nlisinopril / vancomycin\n \nChief...,NaN,NaN,0,0,0
1,21091437.0,2087-05-08 00:05:00,0,0,0,0,43819938,Allergies: \namitriptyline / Sulfa (Sulfonamid...,0.03,cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0
2,28951019.0,2087-05-08 00:05:00,0,0,0,0,44864920,Allergies: \nPenicillins\n \nChief Complaint:\...,NaN,<0.01. cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0
3,21236438.0,2088-04-14 02:58:00,0,0,0,0,47239325,Allergies: \nMULTIPLE - SEE LIST **** / amoxic...,0.02,cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0
4,23821411.0,2089-12-11 00:06:00,0,0,0,0,41010651,Allergies: \nchlorhexidine\n \nChief Complaint...,0.07,cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0


In [7]:
with open('../../data/conversations_new.json') as f:
    conversations = json.load(f)

In [8]:
N = 5  # Number of elements to display

for i, item in enumerate(conversations[:N], start=1):
    print(f"Element {i}:")
    print(json.dumps(item, indent=4))
    print("\n" + "-"*40 + "\n")

Element 1:
{
    "id": 44673611,
    "image": "44673611.jpeg",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nAllergies: \nlisinopril / vancomycin\n \nChief Complaint:\nAltered mental status\n \nMajor Surgical or Invasive Procedure:\nRight AKA ___\n \nHistory of Present Illness:\nMr. ___ is a ___ male with history notable for seizure \ndisorder due to prior CVAs, poorly controlled IDDM complicated \nby ___ ulcers and bilateral transmetatarsal amputations, \nrecurrent cellulitis/osteomyelitis,  CKD stage 3, CAD s/p DESx2 \n___ and recent UTI on amoxcillin/clavulanate who \npresents to the ___ with altered mental status and somnolence.\n\nPatient had numerous admissions at ___ and in the ___ \n___ system for similar presentation. He was last at ___ \nfrom ___ with AMS and was found to have a UTI ___ \npan-sensitive E. Coli. He was also noted to have issues with \nurinary retention during that stay and was discharged with a \nfoley catheter an

In [3]:
with open('../../data/conv_templates.json') as f:
    conv_templates = json.load(f)

In [4]:
df = df[~df['text'].isna()]
df = df[(df['troponin'] & ~df['valuenum'].isnull() & ~df['comments'].isnull()) | (df['troponin'] == 0)]

In [5]:
conversations = []

for idx, row in df.iterrows():
    idx_1 = row['study_id']
    conv = {
        "id": row['study_id'],
        "image": f"{idx_1}.jpeg",
        "conversations": []
    }

    if row['STEMI']:
        conv['diagnosis'] = 'STEMI'
    elif row['NSTEMI']:
        conv['diagnosis'] = 'NSTEMI'
    else:
        conv['diagnosis'] = 'NO ACS'

    conv['conversations'].append({
        "from": "human",
        "value" : random.choice(conv_templates['st_elev_q']).replace('{INSERT PATIENT NOTE}', row['text'])
    })

    if row['st_elevation']:
        conv['conversations'].append({
            "from": "gpt",
            "value" : random.choice(conv_templates['st_elev_a_true'])
        })
        conv['conversations'].append({
            "from": "human",
            "value" : random.choice(conv_templates['troponin_q']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
        })
        if row['troponin']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q_trop']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value']).replace('{INSERT_RESULT}', str(row['valuenum']) + ',' + row['comments'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })

        if row['STEMI']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_stemi'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_normal'])
            })
    else:
        conv['conversations'].append({
            "from": "gpt",
            "value" : random.choice(conv_templates['st_elev_a_false'])
        })
        conv['conversations'].append({
            "from": "human",
            "value" : random.choice(conv_templates['st_depres_twi_q']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
        })
        if row['st_depression'] or row['t_wave']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['st_depres_twi_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['troponin_q']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['st_depres_twi_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['troponin_q']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })

        if row['troponin']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_true'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q_trop']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value']).replace('{INSERT_RESULT}', str(row['valuenum']) + ',' + row['comments'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['troponin_a_false'])
            })
            conv['conversations'].append({
                "from": "human",
                "value" : random.choice(conv_templates['diag_q']).replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })
            
        if row['NSTEMI']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_nstemi'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : random.choice(conv_templates['diag_a_normal'])
            })
    conversations.append(conv)


In [6]:
with open('../../data/conversations_new.json', 'w') as f:
    json.dump(conversations, f)
